In [3]:
# !pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 8.6 MB/s eta 0:00:00m eta 0:00:010:00:01


In [25]:
import pandas as pd
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19 --> Megan's Answer

In [26]:
taxi_jan23_df = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")
taxi_feb23_df = pd.read_parquet("data/yellow_tripdata_2023-02.parquet")

taxi_feb23_df.rename(columns={"Airport_fee": "airport_fee"}, inplace=True)

taxi_df = pd.concat([taxi_jan23_df, taxi_feb23_df]).reset_index(drop=True)

In [27]:
print(taxi_jan23_df.shape)
print(taxi_feb23_df.shape)
print(taxi_df.shape)

(3066766, 19)
(2913955, 19)
(5980721, 19)


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59 <-- Megan's Answer (I got 41.62 as the standard deviation)
* 52.59
* 62.59

In [24]:
# The pickup and dropoff features are already datetime datatypes. 
# To calculate duration we need to subtract pickup time from dropoff time:

taxi_jan23_df["duration"] = taxi_jan23_df["tpep_dropoff_datetime"] - taxi_jan23_df["tpep_pickup_datetime"]

# Convert the duration to minutes
taxi_jan23_df["duration"] = taxi_jan23_df["duration"].dt.seconds / 60

In [28]:
def duration_minutes(df):
    # The pickup and dropoff features are already datetime datatypes. 
    # To calculate duration we need to subtract pickup time from dropoff time:
    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    
    # Convert the duration to minutes
    df["duration"] = df["duration"].dt.seconds / 60

In [29]:
duration_minutes(taxi_jan23_df)

In [30]:
# Now calculate the standard deviation of the trip durations in January
taxi_jan23_df["duration"].std()

41.62919110966266

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98% <-- Megan's Answer

In [38]:
# Dropping outliers
taxi_df_no_outliers = taxi_jan23_df[(taxi_jan23_df["duration"] >= 1) & (taxi_jan23_df["duration"] <= 60)]

In [39]:
# Calculate the percentage of records that remain after outliers are dropped
taxi_df_no_outliers.shape[0]/taxi_jan23_df.shape[0]*100

98.12212604417813

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515 <-- Megan's Answer (actual value calculared = 521)
* 715

In [41]:
# One Hot Encode PULocationID and DOLocationID
X = taxi_jan23_df[["PULocationID", "DOLocationID"]].astype(str)

In [42]:
X_OHE = pd.get_dummies(X)

In [43]:
X_OHE.shape

(3066766, 518)

In [44]:
X.head()

,PULocationID,DOLocationID
0,161,141
1,43,237
2,48,238
3,138,7
4,107,79


In [45]:
taxi_jan23_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [46]:
pd.set_option('display.max_columns', None)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [47]:
# Define train test split
X = X_OHE
y = taxi_jan23_df["duration"]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [48]:
# Instatiate the model:
lr = LinearRegression()

# Train the model:
lr.fit(Xtrain,ytrain)

LinearRegression()

In [49]:
predictions = lr.predict(Xtest)

In [52]:
# Calculate the RMSE of the model on the training data
MSE = mean_squared_error(ytest, predictions)

In [54]:
MSE

1743.3643458254319

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [38]:
RMSE2 = np.sqrt(mean_squared_error(ytest, predictions))

In [39]:
RMSE2

42.84900130286491

## Submit the results

* Submit your results here: https://courses.datatalks.club/mlops-zoomcamp-2024/homework/hw1
* If your answer doesn't match options exactly, select the closest one